---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [2]:
import pandas as pd
import numpy as np

In [3]:
address_df = pd.read_csv('addresses.csv', encoding='ISO-8859-1')

In [4]:
def get_most_important_features(data_train, target_train, n = -1):
    from sklearn.tree import DecisionTreeClassifier
    
    clf = DecisionTreeClassifier(random_state = 0)
    clf.fit(data_train.values, target_train.values)
    features = clf.feature_importances_

    
    df = pd.DataFrame({'feature_name':data_train.columns, 'feature':features})
    df.set_index('feature_name', inplace = True)
    df.sort('feature', inplace=True, ascending=False)
    
    return list(df.index[:n])

In [5]:
'''
    return a Serie with the new category created
'''
def codify(df, column):
    values = df[column].unique()
    codes = {}
    for code, value in zip(range(1,len(values)+1),values):
        codes[value] = code
    
    return df[column].apply(lambda code: codes[code])
#codify(data_train, 'agency_name')

In [6]:
data_test = pd.read_csv('test.csv', encoding='ISO-8859-1')

### Creating Data Train Frame

In [34]:
def get_elaspsed_days(data1, data2):
    if type(data1) == float or type(data2) == float:
        return 0
    
    #2004-03-16 11:40:00
    y1,m1,d1 = data1.split(" ")[0].split('-')
    y2,m2,d2 = data2.split(" ")[0].split('-')
    
    y1 = int(y1)
    y2 = int(y2)
    m1 = int(m1)
    m2 = int(m2)
    d1 = int(d1)
    d2 = int(d2)
    
    dd = 0
    dm = 0
    dy = 0
    
    if m2 > m1:
        dd = 30 - d1 + d2
    else:
        dd = d2 - d1
    
    if y2 > y1:
        dm = 12- m1 + m2
    else:
        dm = m2 - m1 - 1
        if dm < 0:
            dm = 0
            
    if y1 > y2:
        return 0
    else:
        dy = y2 - y1 - 1
        if dy < 0:
            dy = 0        
    
    elapsed_days = dy*365 + dm*30 + dd
    if elapsed_days < 0:
        return 0
    else:
        return elapsed_days
    
date1 = '2004-03-16 11:40:00'
date2 = '2004-05-23 12:30:00'
get_elaspsed_days(date1, date2)

67

In [64]:
data_train = pd.read_csv('train.csv', encoding='ISO-8859-1')

data_train.set_index('ticket_id', inplace=True)

data_train['compliance'].fillna(value=-1, inplace=True)
data_train = data_train[data_train['compliance'] >= 0]

target = data_train['compliance'].copy()

columns_to_del = ['payment_amount','payment_date','payment_status','balance_due', 'collection_status',
                  'grafitti_status', 'clean_up_cost','state_fee','admin_fee','violation_description','zip_code',
                  'mailing_address_str_name', 'violation_zip_code', 'violation_street_name',
                  'violation_street_number','violator_name', 'inspector_name','city','compliance_detail',
                  'compliance']

for column in columns_to_del:
    del data_train[column]

columns_to_code = ['agency_name','country','disposition','violation_code','state','non_us_str_code','mailing_address_str_number']
for column in columns_to_code:
    data_train[column].fillna(value='--', inplace=True)
    data_train[column] = codify(data_train,column)  

data_train['days_elapsed'] = data_train.apply(lambda row: get_elaspsed_days(row['ticket_issued_date'],row['hearing_date']),axis=1)
data_train['days_elapsed'].fillna(value=data_train['days_elapsed'].mean())

del data_train['ticket_issued_date']
del data_train['hearing_date']


/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [65]:
data_train.fillna(value=0,inplace=True)
most_imp_feat = get_most_important_features(data_train,target)

data_train = data_train[most_imp_feat]
#most_imp_feat

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


### Creating Data test frame

In [66]:
columns_only_in_train = ['payment_amount','payment_date' ,'payment_status','balance_due', 'collection_status', 
                         'compliance', 'compliance_detail']

columns_to_del = ['grafitti_status', 'clean_up_cost','state_fee','admin_fee','violation_description','zip_code',
                  'mailing_address_str_name', 'violation_zip_code', 'violation_street_name',
                  'violation_street_number','violator_name', 'inspector_name','city']

data_test = pd.read_csv('test.csv')

data_test.set_index('ticket_id',inplace=True)

for column in columns_to_del:
    del data_test[column]

      
columns_to_code = ['agency_name','country','disposition','violation_code','state','non_us_str_code','mailing_address_str_number']
for column in columns_to_code:
    data_test[column].fillna(value='--', inplace=True)
    data_test[column] = codify(data_test,column)   

data_test['days_elapsed'] = data_test.apply(lambda row: get_elaspsed_days(row['ticket_issued_date'],row['hearing_date']),axis=1)
data_test['days_elapsed'].fillna(value=data_test['days_elapsed'].mean())
del data_test['ticket_issued_date']
del data_test['hearing_date']

data_test = data_test[most_imp_feat]

In [67]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

scaler = MinMaxScaler()
#X_train, X_test, y_train, y_test = train_test_split(data_train.values, target.values, random_state = 0)
X_train, X_test, y_train, y_test = data_train.values, data_test.values, target.values,[]

#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

#print(X_train.shape)
#print(y_train.shape)
#print(data_test.shape)

### Decision Tree approach

In [12]:
'''from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier().fit(X_train, y_train)

print("score: {}".format(clf.score(X_train, y_train)))
#print("score: {}".format(clf.score(X_test, y_test)))

pred = clf.predict(data_test.values)

prediction = pd.Series(pred, index = data_test.index)
prediction
'''


'from sklearn.tree import DecisionTreeClassifier\n\nclf = DecisionTreeClassifier().fit(X_train, y_train)\n\nprint("score: {}".format(clf.score(X_train, y_train)))\n#print("score: {}".format(clf.score(X_test, y_test)))\n\npred = clf.predict(data_test.values)\n\nprediction = pd.Series(pred, index = data_test.index)\nprediction\n'

### Logistic regression approach

In [68]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train, y_train)

print("Score:{}".format(clf.score(X_train, y_train)))

pred_prob = clf.fit(X_train, y_train).predict_proba(X_test)
prediction_prob = pd.Series(pred_prob[:,1],index=data_test.index)



Score:0.9347760820615462


/opt/conda/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


### SVM approach

In [14]:
from sklearn.svm import SVC

'''
gamma = []
C = []
svm_clf = SVC(kernel='rbf', C = 1)
svm_clf.fit(X_train, y_train)

score = svm_clf.decision_function(X_test)
fpr, tpr, _ = roc_curve(y_test, score)
roc_auc = auc(fpr, tpr)
accuracy = svm_clf.score(X_test, y_test)
print("accuracy = {:.2f}   AUC = {:.2f}".format(accuracy, roc_auc))
'''

'\ngamma = []\nC = []\nsvm_clf = SVC(kernel=\'rbf\', C = 1)\nsvm_clf.fit(X_train, y_train)\n\nscore = svm_clf.decision_function(X_test)\nfpr, tpr, _ = roc_curve(y_test, score)\nroc_auc = auc(fpr, tpr)\naccuracy = svm_clf.score(X_test, y_test)\nprint("accuracy = {:.2f}   AUC = {:.2f}".format(accuracy, roc_auc))\n'

### Neural Network approach

In [15]:
#from sklearn.neural_network import MLPClassifier

'''scaler = MinMaxScaler()
#select the main features
layers = [10,10,10]
mlp = MLPClassifier(hidden_layer_sizes = layers, alpha = 5.0, random_state = 0, solver = 'lbfgs')
mlp.fit(X_train, y_train)
'''
#print("Score:{}".format(mlp.score(X_train, y_train)))
#print("Score:{}".format(mlp.score(X_test, y_test)))


#prediction = pd.Series(pred, index=data_test.index)
#prediction


"scaler = MinMaxScaler()\n#select the main features\nlayers = [10,10,10]\nmlp = MLPClassifier(hidden_layer_sizes = layers, alpha = 5.0, random_state = 0, solver = 'lbfgs')\nmlp.fit(X_train, y_train)\n"

In [69]:

def blight_model():
    
    # Your code here
    
    return prediction_prob